## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-14-52-11 +0000,bbc,Belarus frees 123 prisoners as US lifts sanctions,https://www.bbc.com/news/articles/c4gpwlkklyxo...
1,2025-12-13-14-48-43 +0000,cbc,"Organizer arrested after fans rip up seats, th...",https://www.cbc.ca/news/world/soccer-fans-mess...
2,2025-12-13-14-46-08 +0000,nyt,"In Trump’s Justice Dept., Failing in Court Mig...",https://www.nytimes.com/2025/12/12/us/politics...
3,2025-12-13-14-45-01 +0000,nyt,Venezuela Oil Tanker Seized by U.S. Was Part o...,https://www.nytimes.com/2025/12/12/world/ameri...
4,2025-12-13-14-41-04 +0000,bbc,Street art festival sees Lagos turned into 'op...,https://www.bbc.com/news/articles/c4ge4q8gypzo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2419/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,42
402,new,13
155,russia,11
156,ukraine,11
220,year,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
190,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,119
284,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,89
224,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,88
140,2025-12-13-00-06-15 +0000,nyt,New Photos From Epstein Estate Show Ties to Tr...,https://www.nytimes.com/2025/12/12/us/politics...,88
110,2025-12-13-02-16-51 +0000,bbc,"New photos from Epstein estate show Trump, And...",https://www.bbc.com/news/articles/crk777lj763o...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
190,119,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
284,47,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...
135,45,2025-12-13-00-29-23 +0000,nypost,Zelensky makes daring frontline visit to show ...,https://nypost.com/2025/12/12/world-news/zelen...
26,44,2025-12-13-12-55-06 +0000,nypost,"Public urination, drug use, other NYC quality-...",https://nypost.com/2025/12/13/us-news/public-u...
30,42,2025-12-13-12-43-16 +0000,bbc,One million households without power in Ukrain...,https://www.bbc.com/news/articles/c0l9954yr9ko...
256,34,2025-12-12-17-34-00 +0000,wsj,Amtrak managers are giving up $16 million of t...,https://www.wsj.com/politics/policy/amtrak-wor...
217,32,2025-12-12-20-01-34 +0000,latimes,Leaders of secretive California religious grou...,https://www.latimes.com/california/story/2025-...
127,31,2025-12-13-01-09-53 +0000,nyt,House G.O.P. Releases Health Plan That Would A...,https://www.nytimes.com/2025/12/12/us/politics...
83,30,2025-12-13-07-44-36 +0000,nypost,Iran arrests Nobel Peace Prize laureate Narges...,https://nypost.com/2025/12/13/world-news/nobel...
31,29,2025-12-13-12-41-13 +0000,nypost,Outgoing NYC Mayor Eric Adams warns ‘dark days...,https://nypost.com/2025/12/13/us-news/outgoing...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
